In [29]:
import os
import random
import argparse
import logging
import mindspore
import numpy as np
from mindspore import dtype as mstype
import mindspore_hub as mshub
import mindspore.dataset as ds
import mindspore.dataset.vision.c_transforms as C
import mindspore.dataset.transforms.c_transforms as C2
import mindspore.dataset.vision.py_transforms as P
from mindspore.nn import SoftmaxCrossEntropyWithLogits
from mindspore.communication import init
from mindspore.nn import Momentum, Adam, AdamWeightDecay, SGD
from mindspore import Model, context
from mindspore import common, Parameter
from mindspore.context import ParallelMode
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig, LossMonitor
from mindspore import load_checkpoint, load_param_into_net
from resnet import resnet50
import mindspore.nn as nn
from mindspore import Tensor
from mindspore.common import dtype as mstype
from mindspore.nn.loss import LossBase
from mindspore.ops import functional as F
from mindspore.ops import operations as P
from PIL import Image

In [30]:
os.environ["CUDA_VISIBLE_DEVICES"] = "6"
DEVICE_NUM = 1
EPOCH_PER_CKPT = 5
# common.set_seed(1)
random.seed(1)
context.set_context(mode=context.GRAPH_MODE, device_target='GPU')

data_home = '/home/twang/BDCI2021/test'
checkpoint_path_1 = '/home/twang/BDCI2021/checkpoint/ckpt_sgd_mixup_ls/train_resnet_sgd_mixup_ls_1-65_673.ckpt'
checkpoint_path_2 = '/home/twang/BDCI2021/checkpoint/ckpt_transfer/train_resnet_transfer-40_2694.ckpt'
batch_size = 128
epoch_size = 100
num_classes = 2388

In [31]:
def create_dataset(repeat_num=1, training=True):
    
    assert os.path.exists(data_home), "the dataset path is invalid!"
    cifar_ds = ds.ImageFolderDataset(data_home, decode=True, shuffle=False)
 
    resize_height = 224
    resize_width = 224
    rescale = 1.0 / 255.0
    shift = 0.0
    random_horizontal_op = C.RandomHorizontalFlip()
    resize_op = C.Resize((resize_height, resize_width)) # interpolation default BILINEAR
    rescale_op = C.Rescale(rescale, shift)
    normalize_op = C.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    changeswap_op = C.HWC2CHW() 
    cifar_ds = cifar_ds.map(operations=[random_horizontal_op, resize_op, rescale_op, normalize_op, changeswap_op], input_columns=["image"], output_columns=["image"])
    
    # Declare an apply_func function which returns a Dataset object
    def apply_func(data):
        data = data.batch(batch_size, True)
        return data
    cifar_ds = cifar_ds.apply(apply_func)
    return cifar_ds



In [32]:
net1 = resnet50(batch_size, num_classes)
ls1 = SoftmaxCrossEntropyWithLogits(sparse=True, reduction="mean")
opt1 = Momentum(filter(lambda x: x.requires_grad, net1.get_parameters()), 0.01, 0.9)
model1 = Model(net1, loss_fn=ls1, optimizer=opt1, metrics={'acc', 'acc_without_others', 'predict_matrix'})
eval_dataset1 = create_dataset(training=False)

In [33]:
param_dict_1 = load_checkpoint(checkpoint_path_1)
load_param_into_net(net1, param_dict_1)

[]

In [34]:
res1 = model1.eval(eval_dataset1)

libpng warning: iCCP: extra compressed data
libpng warning: iCCP: extra compressed data
libpng warning: iCCP: extra compressed data
libpng warning: iCCP: extra compressed data
libpng warning: iCCP: extra compressed data
libpng warning: iCCP: extra compressed data
libpng warning: iCCP: extra compressed data
libpng warning: iCCP: extra compressed data


In [35]:
res1

{'acc': 0.9629159902597403,
 'acc_without_others': 0.9689782873376623,
 'predict_matrix': [[array([19.097279, 10.604555,  2.01472 , ...,  3.788072, -2.682207,
          -4.786611], dtype=float32),
   0],
  [array([19.090378 , 11.846951 ,  4.338558 , ...,  0.6888144, -1.9356458,
          -3.8041613], dtype=float32),
   0],
  [array([18.124533  ,  8.25824   ,  1.9522393 , ...,  4.1076508 ,
          -0.95258224, -0.459672  ], dtype=float32),
   0],
  [array([ 6.6348767 , 14.160664  ,  7.33212   , ...,  0.21011856,
          -2.6063435 ,  0.6673126 ], dtype=float32),
   1],
  [array([ 2.290106  , 10.822311  ,  4.68204   , ..., -0.41767314,
          -1.8404096 , -2.1053848 ], dtype=float32),
   1],
  [array([ 6.631244 , 11.435616 ,  3.227872 , ...,  1.1473536, -3.9048657,
          -3.1837614], dtype=float32),
   1],
  [array([ 4.441927 , 13.193056 ,  5.532023 , ..., -2.4328866, -1.023249 ,
          -1.8123313], dtype=float32),
   1],
  [array([ 5.0992603,  7.439384 , 13.493286 , ..., -

In [36]:
hub_model = "mindspore/ascend/1.2/resnet50thorcp_v1.2_imagenet2012"
net2 = mshub.load(hub_model, class_num=2388, force_reload=False)
param_dict_2 = load_checkpoint("/home/twang/.mscache/mindspore/ascend/1.2/resnet50thorcp_ascend_v120_imagenet2012_official_cv_bs256_acc76.ckpt")
wei = Parameter(Tensor(np.ones((2388, 2048)), mindspore.float32), name="moments.end_point.weight", requires_grad=True)
bia = Parameter(Tensor(np.ones((2388)), mindspore.float32), name="moments.end_point.bias", requires_grad=True)
param_dict_2['end_point.weight']=wei
param_dict_2['end_point.bias']=bia
load_param_into_net(net2, param_dict_2)
net2.set_train(True)
ls2 = SoftmaxCrossEntropyWithLogits(sparse=True, reduction="mean")
opt2 = Momentum(filter(lambda x: x.requires_grad, net2.get_parameters()), 0.01, 0.9)
model2 = Model(net2, loss_fn=ls2, optimizer=opt2, metrics={'acc', 'acc_without_others', 'predict_matrix'})
eval_dataset2 = create_dataset(training=False)

Checking /home/twang/.mscache/mindspore/ascend/1.2/resnet50thorcp_v1.2_imagenet2012.md...Passed!
Checking /home/twang/.mscache/mindspore/ascend/1.2/resnet50thorcp_v1.2_imagenet2012.md...Passed!


In [37]:
param_dict_2 = load_checkpoint(checkpoint_path_2)
load_param_into_net(net2, param_dict_2)
res2 = model2.eval(eval_dataset2)

libpng warning: iCCP: extra compressed data
libpng warning: iCCP: extra compressed data
libpng warning: iCCP: extra compressed data
libpng warning: iCCP: extra compressed data
libpng warning: iCCP: extra compressed data
libpng warning: iCCP: extra compressed data
libpng warning: iCCP: extra compressed data
libpng warning: iCCP: extra compressed data


In [38]:
res2

{'acc': 0.9719967532467533,
 'acc_without_others': 0.9762834821428571,
 'predict_matrix': [[array([1839.2665, 1831.6179, 1818.4619, ..., 1814.7048, 1806.5464,
          1808.5974], dtype=float32),
   0],
  [array([1828.4786, 1820.0261, 1809.8417, ..., 1804.8982, 1797.9878,
          1798.1987], dtype=float32),
   0],
  [array([2208.2324, 2196.3442, 2186.0132, ..., 2183.2942, 2173.0889,
          2178.061 ], dtype=float32),
   0],
  [array([1841.6486, 1854.3196, 1842.2007, ..., 1829.5452, 1824.2097,
          1826.1232], dtype=float32),
   1],
  [array([2586.674 , 2590.902 , 2588.977 , ..., 2582.2368, 2575.9497,
          2578.8452], dtype=float32),
   1],
  [array([2161.2114, 2171.2798, 2152.8367, ..., 2144.4868, 2135.1404,
          2137.2383], dtype=float32),
   1],
  [array([1703.3276, 1716.2001, 1705.7686, ..., 1692.6954, 1686.9805,
          1688.5359], dtype=float32),
   1],
  [array([1751.2925, 1754.4098, 1765.7065, ..., 1747.1372, 1741.1917,
          1740.4742], dtype=float32)

In [39]:
def softmax(y_pred):
	max = np.max(y_pred)
	softmax = np.exp(y_pred-max) / sum(np.exp(y_pred-max))
	return softmax

In [41]:
predict_matrix_1 = res1['predict_matrix']
predict_matrix_2 = res2['predict_matrix']
total_num = len(predict_matrix_1)
acc_list = []

for p in range(10):
	correct_num = 0
	current_num = 0
	for i in range(total_num):
		ans = predict_matrix_1[i][1]
		prob_array_1 = softmax(predict_matrix_1[i][0])
		prob_array_2 = softmax(predict_matrix_2[i][0])
		# print(prob_array_1)
		if (np.max(prob_array_1) < 0.995 and np.max(prob_array_2) < 0.995):
			result = 21
		else:
			prob_array = (p / 10) * prob_array_1 + (1 - p / 10) * prob_array_2
			result = np.where(prob_array == np.max(prob_array))
		# print('result: ', result)
		# print('ans: ', ans)
		if (result == ans):
			correct_num = correct_num + 1
		current_num = current_num + 1
		if (current_num % 10000 == 0):
			print(current_num, "/", total_num)
	print('acc: ', correct_num / total_num)
	acc_list.append(correct_num / total_num)

print('acc_list: ', acc_list)
print('acc_max: ', max(acc_list))

10000 / 39424
20000 / 39424
30000 / 39424
acc:  0.94140625
10000 / 39424
20000 / 39424
30000 / 39424
acc:  0.94140625
10000 / 39424
20000 / 39424
30000 / 39424
acc:  0.94140625
10000 / 39424
20000 / 39424
30000 / 39424
acc:  0.94140625
10000 / 39424
20000 / 39424
30000 / 39424
acc:  0.94140625
10000 / 39424
20000 / 39424
30000 / 39424
acc:  0.94140625
10000 / 39424
20000 / 39424
30000 / 39424
acc:  0.9407467532467533
10000 / 39424
20000 / 39424
30000 / 39424
acc:  0.9399857954545454
10000 / 39424
20000 / 39424
30000 / 39424
acc:  0.9384385146103896
10000 / 39424
20000 / 39424
30000 / 39424
acc:  0.9366122159090909
acc_list:  [0.94140625, 0.94140625, 0.94140625, 0.94140625, 0.94140625, 0.94140625, 0.9407467532467533, 0.9399857954545454, 0.9384385146103896, 0.9366122159090909]
acc_max:  0.94140625
